<a href="https://colab.research.google.com/github/june-oh/cseg109/blob/main/LAB3_RNN_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB3

## torch.nn.Embedding

In [30]:

train_data = '오패산터널 총격전 용의자 검거'

word_set = set(train_data.split())

vocab = {word: i+2 for i, word in enumerate(word_set)}
vocab['<unk>'] = 0
vocab['<pad>'] = 1
print(vocab)

{'오패산터널': 2, '총격전': 3, '용의자': 4, '검거': 5, '<unk>': 0, '<pad>': 1}


In [31]:
import torch

embedding_dim=8
embedding_table = torch.rand((len(vocab),embedding_dim))
embedding_table

tensor([[0.2583, 0.8338, 0.7550, 0.4972, 0.7454, 0.0163, 0.0556, 0.5926],
        [0.8886, 0.4207, 0.9760, 0.8180, 0.4810, 0.3890, 0.2459, 0.9274],
        [0.2370, 0.1464, 0.1874, 0.0617, 0.9903, 0.5757, 0.0461, 0.7934],
        [0.1399, 0.3317, 0.5138, 0.5391, 0.9422, 0.0507, 0.3209, 0.5720],
        [0.9781, 0.9840, 0.7835, 0.3799, 0.6893, 0.5994, 0.1305, 0.9253],
        [0.5010, 0.8860, 0.1312, 0.0888, 0.6208, 0.4741, 0.7910, 0.1973]])

In [32]:
sample = "용의자 총격전 끝 검거".split()
sample

['용의자', '총격전', '끝', '검거']

In [33]:
idxes = []

# 각 단어를 정수로 변환
for word in sample:
  try:
    idxes.append(vocab[word])
  except KeyError:
    idxes.append(vocab['<unk>'])
idxes = torch.LongTensor(idxes)

lookup_result = embedding_table[idxes, :]
print(lookup_result)

tensor([[0.9781, 0.9840, 0.7835, 0.3799, 0.6893, 0.5994, 0.1305, 0.9253],
        [0.1399, 0.3317, 0.5138, 0.5391, 0.9422, 0.0507, 0.3209, 0.5720],
        [0.2583, 0.8338, 0.7550, 0.4972, 0.7454, 0.0163, 0.0556, 0.5926],
        [0.5010, 0.8860, 0.1312, 0.0888, 0.6208, 0.4741, 0.7910, 0.1973]])


## Download text Data

In [34]:
import urllib.request

urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")

('2016-10-20.txt', <http.client.HTTPMessage at 0x7ff56afdac80>)

In [35]:
texts = [_.strip() for _ in open('2016-10-20.txt').readlines()]

In [36]:
texts[:4]

['',
 '19  1990  52 1 22',
 '오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽

# MLP Uni-Char LM
-  오->패
- 패->산
- 산->터
- 널->
- ->총
- 총->격

In [37]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import re
from torch import optim

class CharDataset(Dataset):
    def __init__(self, text):
        self.characters = sorted(list(set(text)))
        self.char_to_int = dict((c, i) for i, c in enumerate(self.characters))
        self.int_to_char = dict(enumerate(self.characters))
        self.int_text = np.array([self.char_to_int[char] for char in text])

    def __len__(self):
        return len(self.int_text)-1

    def __getitem__(self, idx):
        return self.int_text[idx],self.int_text[idx+1]

In [38]:
full_text = ' '.join([sentence.strip() for sentence in texts])
full_text = re.sub(r'\s+', ' ', full_text)  # 여러 개의 공백을 하나의 공백으로 줄임

dataset = CharDataset(full_text)
data_loader = DataLoader(dataset, batch_size=32, shuffle=False)

In [39]:
next(iter(data_loader))

[tensor([   0,    2,   10,    0,    2,   10,   10,    1,    0,    6,    3,    0,
            2,    0,    3,    3,    0, 1152, 1667,  891, 1591,  289,    0, 1456,
           85, 1273,    0, 1185, 1231, 1247,    0,   71]),
 tensor([   2,   10,    0,    2,   10,   10,    1,    0,    6,    3,    0,    2,
            0,    3,    3,    0, 1152, 1667,  891, 1591,  289,    0, 1456,   85,
         1273,    0, 1185, 1231, 1247,    0,   71,   66])]

In [40]:
class UniCharLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(UniCharLM, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        #적절한 layer를 추가하세요.
        self.linear = nn.Linear(embedding_dim, vocab_size)


    def forward(self, inputs):
        #forward 함수를 작성하세요.
        embeds = self.embeddings(inputs)
        out = self.linear(embeds)
        return out


## CPU 학습

In [42]:
from tqdm.auto import tqdm

vocab_size = len(dataset.characters)
# 모델 인스턴스 생성
embedding_dim = 128  # 예시로 설정한 임베딩 차원
model = UniCharLM(vocab_size, embedding_dim)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in tqdm(range(1)):
    total_loss = 0
    for batch in tqdm(data_loader, total=len(data_loader),leave=True):
        x,y = batch
        model.zero_grad()
        output = model(x)
        loss = loss_function(output, y)
        loss.backward()
        optimizer.step()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/559121 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

## GPU 학습

In [43]:
from tqdm.auto import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data_loader = DataLoader(dataset, batch_size=1024*10, shuffle=False) #batch 사이즈 변경


vocab_size = len(dataset.characters)
# 모델 인스턴스 생성
embedding_dim = 128  # 예시로 설정한 임베딩 차원
model = UniCharLM(vocab_size, embedding_dim).to(device)  #to cuda

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in tqdm(range(1)):
    total_loss = 0
    for batch in tqdm(data_loader, total=len(data_loader),leave=True):
        x,y = batch
        x = x.to(device)  #to cuda
        y = y.to(device)  #to cuda
        model.zero_grad()
        output = model(x)
        loss = loss_function(output, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch}: Total Loss {total_loss/len(data_loader)}')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1748 [00:00<?, ?it/s]

Epoch 0: Total Loss 3.865399718148212


## Inference

In [44]:
def generate_sentence(model, start_token, word_to_idx, idx_to_word, max_length=20):
    """
    문장 생성 함수
    :param model: 학습된 모델
    :param start_token: 문장 생성을 시작할 첫 단어
    :param word_to_ix: 단어를 인덱스로 변환하는 사전
    :param ix_to_word: 인덱스를 단어로 변환하는 사전
    :param max_length: 생성할 문장의 최대 길이
    :return: 생성된 문장
    """
    current_word = start_token
    sentence = [current_word]
    model.to('cpu')
    model.eval()  # 모델을 추론 모드로 설정

    with torch.no_grad():  # 그래디언트 계산 비활성화
        for _ in range(max_length - 1):
            #auto regressive 하게 입력이 바뀌도록 작성하세요.
            #mystery = 현재입력
            mystery =  word_to_idx[current_word]
            token_index = torch.tensor([mystery], dtype=torch.long)
            output = model(token_index)
            next_word_idx = output.argmax(1).item()
            next_word = idx_to_word[next_word_idx]

            sentence.append(next_word)
            current_word = next_word

    return ' '.join(sentence)

# 인덱스를 단어로 변환하는 사전 생성
idx_to_word = {i: word for word, i in dataset.char_to_int.items()}

In [45]:
start_token = '나'  # 시작 단어 설정
generated_sentence = generate_sentence(model, start_token, dataset.char_to_int, idx_to_word)
print(generated_sentence)

나   이   이   이   이   이   이   이   이   이  


## RNN Charater-level LM

<img src="https://github.com/june-oh/cseg109/blob/main/img/charRNN.png?raw=true" width=80%>

## Torch RNN

`torch.nn.RNN`을 통해서 RNN을 쉽게 구현가능

[https://pytorch.org/docs/stable/generated/torch.nn.RNN.html](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html)


In [46]:
??torch.nn.RNN

In [47]:
input_size =5
hidden_size = 8
rnn_cell = torch.nn.RNN(input_size,hidden_size,batch_first=True)

In [48]:
#sample input
batch_size=2
seq_len = 4
inputs = torch.Tensor(batch_size,seq_len,input_size)

In [49]:
# batch, time, input_size
inputs.shape

torch.Size([2, 4, 5])

In [50]:
rnn_cell(inputs)

(tensor([[[-1.0000e+00, -1.0000e+00, -1.0000e+00,  1.0000e+00, -1.0000e+00,
           -1.0000e+00,  1.0000e+00,  1.0000e+00],
          [ 4.1161e-01, -6.8701e-01, -4.1203e-01, -5.1020e-01,  7.0279e-01,
           -2.4230e-01, -1.4982e-01,  7.0957e-01],
          [ 2.6313e-01,  7.8757e-02, -4.3695e-01, -3.0192e-01,  4.3814e-01,
           -2.3673e-01, -5.2155e-02, -1.6552e-01],
          [ 1.1213e-01,  4.3240e-01, -3.3843e-01, -2.4917e-01,  6.4115e-02,
           -5.1312e-02,  4.1085e-02, -3.1281e-01]],
 
         [[ 1.7738e-01,  2.0837e-01, -2.3067e-01,  8.8792e-04,  1.1042e-01,
           -2.4393e-01,  1.7072e-02, -3.2580e-01],
          [ 7.5236e-02,  4.1236e-01, -2.6619e-01, -2.0103e-01, -2.4033e-02,
           -5.6001e-02,  8.3494e-02, -3.3268e-01],
          [ 5.5591e-02,  4.7387e-01, -3.1139e-01, -1.5089e-02, -7.1949e-02,
           -9.4929e-02,  9.0793e-02, -4.1448e-01],
          [ 4.5988e-02,  4.7004e-01, -2.9447e-01, -5.3693e-02, -1.0396e-01,
           -4.1800e-02,  9.4725e

In [51]:
output, state = rnn_cell(inputs)

In [52]:
output.shape   #batch_size,time_step,hidden_size

torch.Size([2, 4, 8])

In [53]:
state.shape #num_layer, batch_size, hidden_size

torch.Size([1, 2, 8])

In [54]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import re


In [55]:

# 문자 단위 데이터셋 클래스 정의
class CharDataset(Dataset):
    def __init__(self, text, sequence_length=5):
        self.sequence_length =   sequence_length + 1 # 예측 대상 포함
        self.characters = sorted(list(set(text)))
        self.char_to_int = dict((c, i) for i, c in enumerate(self.characters))
        self.int_to_char = dict(enumerate(self.characters))
        self.int_text = np.array([self.char_to_int[char] for char in text])
        self.data = self._create_dataset()

    def _create_dataset(self):
        data = []
        for i in range(0, len(self.int_text) - self.sequence_length,self.sequence_length):
            seq = self.int_text[i:i + self.sequence_length]
            data.append((seq[:-1], seq[1:]))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# 전체 텍스트 데이터를 하나의 문자열로 결합
# 여기서 processed_and_split_dataset은 전처리된 Hugging Face 데이터셋
full_text = ' '.join([sentence.strip() for sentence in texts])
full_text = re.sub(r'\s+', ' ', full_text)  # 여러 개의 공백을 하나의 공백으로 줄임

full_text = re.sub(r's+', ' ', full_text)
dataset = CharDataset(full_text)
data_loader = DataLoader(dataset, batch_size=512, shuffle=False)

In [56]:
print(len(dataset.characters))

1847


In [57]:
import torch.nn as nn



# RNN 모델 클래스 정의
class CharRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers, dropout=0.0):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.GRU(hidden_size, hidden_size, num_layers, dropout=dropout,batch_first=True)
        #self.rnn = nn.RNN(hidden_size, hidden_size, num_layers, dropout=dropout,batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden=None):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded, hidden)
        output = self.fc(output.reshape(-1, self.hidden_size))
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size,self.hidden_size).to(device)



In [58]:
from tqdm.auto import tqdm
vocab_size = len(dataset.characters)
hidden_size = 256
num_layers = 3
model = CharRNN(vocab_size, hidden_size, num_layers).to(device)
def train(model, data_loader, epochs=10, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.train()

    for epoch in tqdm(range(epochs)):


        for batch, (seq_in, seq_out) in tqdm(enumerate(data_loader),total=len(data_loader),leave=False):
            seq_in, seq_out = seq_in.to(device), seq_out.to(device)
            hidden = model.init_hidden(seq_in.shape[0])
            optimizer.zero_grad()

            output, hidden = model(seq_in.long(), hidden)
            hidden =  hidden.detach()  # 중요: 역전파에서 이전 히든 스테이트를 분리

            loss = criterion(output.view(-1, vocab_size), seq_out.view(-1).long())
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

train(model, data_loader, epochs=20)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/5825 [00:00<?, ?it/s]

Epoch [1/20], Loss: 3.8587


  0%|          | 0/5825 [00:00<?, ?it/s]

Epoch [2/20], Loss: 3.6405


...

Epoch [18/20], Loss: 3.0494


  0%|          | 0/5825 [00:00<?, ?it/s]

Epoch [19/20], Loss: 3.0706


  0%|          | 0/5825 [00:00<?, ?it/s]

Epoch [20/20], Loss: 3.0640


# Auto regressive Inference 함수 작성
- torch.argmax를 이용

문자열이 들어오면 다음 문자를 예측하는 함수를 RNN model에 맞게 작성하세요.

## torch.argmax
1D

In [59]:
sample = torch.tensor([1,2,3,4,3,2,1])
torch.argmax(sample, dim=0)

tensor(3)

2D

In [60]:
sample = torch.tensor([[1,2,3,4,4,3,2],
                                    [2,3,4,5,3,2,1]])
torch.argmax(sample, dim=0)

tensor([1, 1, 1, 1, 0, 0, 0])

In [61]:
torch.argmax(sample, dim=1)

tensor([3, 3])

In [62]:
torch.argmax(sample, dim=-1)

tensor([3, 3])

In [81]:
def generate_text(model, start_string, generation_length=100):
    # 모델의 현재 장치를 저장
    original_device = next(model.parameters()).device

    # 인퍼런스를 위해 모델을 CPU로 이동
    model.to('cpu')
    model.eval()

    input_eval = [dataset.char_to_int[c] for c in start_string]
    input_eval = torch.LongTensor(input_eval).unsqueeze(0)

    hidden = model.init_hidden(1).to('cpu')
    text_generated = []

    with torch.no_grad():
        for _ in range(generation_length):
            output, hidden = model(input_eval, hidden)
            predicted_id = torch.argmax(output, dim=-1)[-1]
            input_eval = predicted_id.unsqueeze(0).unsqueeze(0)
            text_generated.append(dataset.int_to_char[predicted_id.item()])

    # 인퍼런스가 끝난 후, 모델을 원래의 장치로 복귀
    model.to(original_device)

    return start_string + ''.join(text_generated)


# 텍스트 생성 예시
print(generate_text(model, start_string="대한민국", generation_length=60))


대한민국 행복교육 시장 관련 지역 남양주 지역 시장 시작 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
